<a href="https://colab.research.google.com/github/Nikhil-24K/GenerativeAI/blob/main/Lab_7__GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

600901/600901 [==============================] - 1s 1us/step
corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...


Epoch 1/60
1565/1565 [==============================] - ETA: 0s - loss: 2.5782
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "nity to so emphasize the unworthiness, s"
nity to so emphasize the unworthiness, so s and at of the sore the the the the the the the the the the the the the the hererent an the the the the here the the the the the the the the whe mand and and and and and and the the the sere the the the hich the the mand and and in the the the the the the the the the sere the the whe the the the he the the the the the the the whe the the the the sore the here the the the the the the the the the
----- diversity: 0.5
----- Generating with seed: "nity to so emphasize the unworthiness, s"
nity to so emphasize the unworthiness, som ande the chathe the pititinig destice the ald an and bowes ing and war is the nonge the ther have fore the tho he ther an herthe the the wore the the the al the withe sos ing isd is mastien and ind the ard at an the 

In [ ]:
!pip install ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.9 MB/s eta 0:00:00


In [ ]:
'''
Using 1D Convs for generate text from rom Nietzsche's and Shakepear's
writings
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
import numpy as np
import random
import sys
import io

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

# 2nd Dataset : Combined works of Shakespeare
# path = get_file('shakespeare_input.txt', origin='https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt')

with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# print(x.shape[0])
# print(x.shape[1])
# print(x.shape[2])


def build_model(maxlen, chars):

    print('Build model...')
    model = Sequential()
    model.add(Conv1D(128, 5, padding='same', activation='relu', input_shape=(maxlen, len(chars))))
    model.add(Flatten())
    model.add(Dense(len(chars), activation='softmax'))

    # model 2 : VGG net like architecture, 5 layers
#     model.add(Conv1D(64, 3, activation='relu',input_shape=(maxlen, len(chars))))
#     model.add(Conv1D(64, 3, activation='relu'))
#     model.add(MaxPooling1D(pool_size=2))
#     model.add(Conv1D(128, 3, activation='relu'))
#     model.add(Conv1D(128, 3, activation='relu'))
#     model.add(MaxPooling1D(pool_size=2))
#     model.add(Flatten())
#     model.add(Dense(len(chars), activation='softmax'))

    #model 3: adapted from https://keras.io/getting-started/sequential-model-guide/
#     model.add(Conv1D(64, 3, activation='relu',input_shape=(maxlen, len(chars))))
#     model.add(Conv1D(64, 3, activation='relu'))
#     model.add(MaxPooling1D(pool_size=3))
#     model.add(Conv1D(128, 3, activation='relu'))
#     model.add(Conv1D(128, 3, activation='relu'))
#     model.add(MaxPooling1D(pool_size=3))
#     model.add(GlobalAveragePooling1D())
#     model.add(Dropout(0.5))
# #     model.add(Flatten())
#     model.add(Dense(len(chars), activation='sigmoid'))


    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

if __name__ == '__main__':
    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

    model = build_model(maxlen, chars)

    import ipdb;

    model.fit(
        x,
        y,
        batch_size=128,
        epochs=60,
        callbacks=[print_callback]
    )
    ipdb.set_trace()

corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...


Build model...
Epoch 1/60
1565/1565 [==============================] - ETA: 0s - loss: 2.3295
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " that
every man is conceived and born in"
 that
every man is conceived and born in the conded of the pored of the conding the stous of the proting of the sure of and of the of the gored to the wist and the condento the condes of the condice the sous the sument ind in the persent of the conse the ded of the god the condestion of the conders of the conded to the conder the condesto the soperstone the stous to the susting of the perions of the conder to the soment as in the corsed
----- diversity: 0.5
----- Generating with seed: " that
every man is conceived and born in"
 that
every man is conceived and born ind and stines londed and dister to sought and and of the pore, in saries indelf
bed them nos and demalisted of whe enture and bean sught so the sould and and inged and and thereresenst one whol the soment 